# EDA totale

Importiamo i pacchetti

In [1]:
import numpy as np

import pandas as pd
import geopandas as gp
from shapely.geometry import Polygon
import json

import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import plotly.graph_objects as go

definiamoci il path ai dati per il mio computer

In [2]:
data_path = "C:/Users/ADMIN/Desktop/Data Science/Lavandini/data/"
file_path_save = data_path + "save_data/"
file_path_proc = "C:/Users/ADMIN/Desktop/Data Science/Lavandini/data/processed/"
file_path_raw = "C:/Users/ADMIN/Desktop/Data Science/Lavandini/data/data_trentino/raw/"
file_path_ext = "C:/Users/ADMIN/Desktop/Data Science/Lavandini/data/data_trentino/external/"

---
## Dati Meteo
Andiamo ora a visualizzare i dati meteo e organizziamoli in un dataframe che saranno poi utili in fase ti classificazione e regressione.

### Precipitazioni
Per prima cosa andiamo a creare e visualizzare il dataframe con le precipitazioni:  

In [3]:
# Importiamo il file csv
tags = ["DATE", "SQUAREID", "RAIN"]
precip_df = pd.read_csv(file_path_raw + 'precipitation-trentino.csv', names=tags)
del(tags)


Aggiustiamo la colonna **DATE** similmente a come già fatto in precedenza per `set_df`

In [4]:
precip_df["DATE"] = pd.to_datetime(precip_df["DATE"], format="%Y%m%d%H%M")
precip_df["MONTH"] = pd.to_datetime(precip_df["DATE"]).dt.month
precip_df["DAY"] = pd.to_datetime(precip_df["DATE"]).dt.day
precip_df["MINUTE"] = pd.to_datetime(precip_df["DATE"]).dt.hour * 60 + pd.to_datetime(precip_df["DATE"]).dt.minute
precip_df["WEEK_DAY"] = pd.to_datetime(precip_df["DATE"]).dt.day_of_week

# ordiniamoli in base alla data
precip_df = precip_df.sort_values(by='DATE')
precip_df = precip_df.reset_index()
del precip_df["index"]

precip_df

,DATE,SQUAREID,RAIN,MONTH,DAY,MINUTE,WEEK_DAY
0,2013-11-01 00:00:00,2383,1,11,1,0,4
1,2013-11-01 00:00:00,5429,2,11,1,0,4
2,2013-11-01 00:00:00,5205,1,11,1,0,4
3,2013-11-01 00:00:00,6754,1,11,1,0,4
4,2013-11-01 00:00:00,7436,1,11,1,0,4
...,...,...,...,...,...,...,...
3489412,2013-12-26 21:10:00,5193,1,12,26,1270,3
3489413,2013-12-26 21:10:00,4967,1,12,26,1270,3
3489414,2013-12-26 21:10:00,4612,1,12,26,1270,3
3489415,2013-12-26 21:10:00,9697,1,12,26,1270,3


In [5]:
i = 2307596
precip_df[i:i+4]

,DATE,SQUAREID,RAIN,MONTH,DAY,MINUTE,WEEK_DAY
2307596,2013-11-30 01:40:00,9582,1,11,30,100,5
2307597,2013-11-30 01:40:00,7225,1,11,30,100,5
2307598,2013-12-02 00:50:00,9033,1,12,2,50,0
2307599,2013-12-02 00:50:00,9866,1,12,2,50,0


Si può vedere che per alcuni orari mancano dei dati, questo è perché...

Aggiungiamo anche il file `precipitation-trentino-data-availability.csv` così da avere dei valori `true` e `false` se i dati sono presenti o mancanti.

In [6]:
# Importiamo il file csv
tags = ["DATE", "IS_RAIN"]
precip_avail_df = pd.read_csv(file_path_raw + 'precipitation-trentino-data-availability.csv', names=tags)
del(tags)

# Aggiustiamo la data
precip_avail_df["DATE"] = pd.to_datetime(precip_avail_df["DATE"], format="%Y%m%d%H%M")

precip_avail_df

,DATE,IS_RAIN
0,2013-11-01 00:00:00,True
1,2013-11-01 00:10:00,True
2,2013-11-01 00:20:00,True
3,2013-11-01 00:30:00,True
4,2013-11-01 00:40:00,True
...,...,...
8779,2013-12-31 23:10:00,False
8780,2013-12-31 23:20:00,False
8781,2013-12-31 23:30:00,False
8782,2013-12-31 23:40:00,False


Ora uniamo i due dataframe così da avere un quadro completo  
Inoltre gestiamo i NaN nelle colonne SQUAREID e RAIN, sostituiamoli con degli zeri

In [7]:
precip_df = pd.merge(precip_df, precip_avail_df,on="DATE", how="right")

precip_df["MONTH"] = pd.to_datetime(precip_df["DATE"]).dt.month
precip_df["DAY"] = pd.to_datetime(precip_df["DATE"]).dt.day
precip_df["MINUTE"] = pd.to_datetime(precip_df["DATE"]).dt.hour * 60 + pd.to_datetime(precip_df["DATE"]).dt.minute
precip_df["WEEK_DAY"] = pd.to_datetime(precip_df["DATE"]).dt.day_of_week

# ordiniamoli in base alla data
precip_df = precip_df.sort_values(by='DATE')
precip_df = precip_df.reset_index()
del precip_df["index"]

# gestiamo i NaN nelle colonne SQUAREID e RAIN, sostituiamoli con uno 0
precip_df['SQUAREID'] = precip_df['SQUAREID'].fillna(0).astype(np.int64)
precip_df['RAIN'] = precip_df['RAIN'].fillna(0).astype(np.int64)
del precip_df["DATE"]

precip_df

,SQUAREID,RAIN,MONTH,DAY,MINUTE,WEEK_DAY,IS_RAIN
0,2383,1,11,1,0,4,True
1,7930,1,11,1,0,4,True
2,7559,1,11,1,0,4,True
3,9815,2,11,1,0,4,True
4,6140,1,11,1,0,4,True
...,...,...,...,...,...,...,...
3490952,0,0,12,31,1390,1,False
3490953,0,0,12,31,1400,1,False
3490954,0,0,12,31,1410,1,False
3490955,0,0,12,31,1420,1,False


---

# Creiamo un dataframe totate

Carichiamo i dataframe `power_time_nov_df` e `power_time_dec_df` dalla cartella _save_data_:


In [8]:
power_time_nov_df = pd.read_csv(file_path_save + "power_time_nov_df.csv")
power_time_dec_df = pd.read_csv(file_path_save + "power_time_dec_df.csv")
power_time_dec_df

,DAY,MINUTE,WEEK_DAY,SQUAREID,W_POWER,MONTH
0,1,0,6,4037,31726.077408,12
1,1,0,6,4154,31726.077408,12
2,1,0,6,4155,31726.077408,12
3,1,0,6,4156,31726.077408,12
4,1,0,6,4269,31726.077408,12
...,...,...,...,...,...,...
11298379,31,1430,1,8502,65469.601728,12
11298380,31,1430,1,8503,65469.601728,12
11298381,31,1430,1,8621,21823.200576,12
11298382,31,1430,1,8972,65469.601728,12


Uniamo i due dataframe e facciamo un merge con `precip_df`

In [10]:
temp_df = pd.concat([power_time_nov_df, power_time_dec_df], ignore_index=True)

final_df = pd.merge(precip_df, temp_df, on=["SQUAREID", "MONTH", "DAY", "MINUTE", "WEEK_DAY"], how="right")
del temp_df

final_df

,SQUAREID,RAIN,MONTH,DAY,MINUTE,WEEK_DAY,IS_RAIN,W_POWER
0,4037,NaN,11,1,0,4,NaN,32348.159136
1,4154,NaN,11,1,0,4,NaN,32348.159136
2,4155,NaN,11,1,0,4,NaN,32348.159136
3,4156,NaN,11,1,0,4,NaN,32348.159136
4,4269,NaN,11,1,0,4,NaN,32348.159136
...,...,...,...,...,...,...,...,...
22227259,8502,NaN,12,31,1430,1,NaN,65469.601728
22227260,8503,NaN,12,31,1430,1,NaN,65469.601728
22227261,8621,NaN,12,31,1430,1,NaN,21823.200576
22227262,8972,NaN,12,31,1430,1,NaN,65469.601728


In [11]:
precip_df[precip_df["SQUAREID"] == 4037]

,SQUAREID,RAIN,MONTH,DAY,MINUTE,WEEK_DAY,IS_RAIN
15758,4037,1,11,2,180,5,True
17210,4037,1,11,2,200,5,True
26963,4037,1,11,2,350,5,True
29329,4037,1,11,2,370,5,True
36851,4037,1,11,2,430,5,True
...,...,...,...,...,...,...,...
3403050,4037,1,12,26,830,3,True
3407537,4037,1,12,26,840,3,True
3410521,4037,1,12,26,850,3,True
3414719,4037,1,12,26,860,3,True


Come da consegna consideriamo solo i giorni feriali:

In [30]:
final_df = final_df[final_df['WEEK_DAY'].isin([0, 1, 2, 3, 4])]
final_df

,SQUAREID,RAIN,MONTH,DAY,MINUTE,WEEK_DAY,IS_RAIN,W_POWER,fascia_oraria
0,4037,NaN,11,1,0,4,NaN,32348.159136,00-8
1,4154,NaN,11,1,0,4,NaN,32348.159136,00-8
2,4155,NaN,11,1,0,4,NaN,32348.159136,00-8
3,4156,NaN,11,1,0,4,NaN,32348.159136,00-8
4,4269,NaN,11,1,0,4,NaN,32348.159136,00-8
...,...,...,...,...,...,...,...,...,...
22227259,8502,NaN,12,31,1430,1,NaN,65469.601728,19-00
22227260,8503,NaN,12,31,1430,1,NaN,65469.601728,19-00
22227261,8621,NaN,12,31,1430,1,NaN,21823.200576,19-00
22227262,8972,NaN,12,31,1430,1,NaN,65469.601728,19-00


Suddividiamo in fasce orarie:

In [32]:
def fascia_oraria(ora):
    if ora>=8 and ora<=12:
        return '8-12'
    elif ora>=12 and ora<=14:
        return '12-14'
    elif ora>=14 and ora<=19:
        return '14-19'
    elif ora>=19:
        return '19-00'
    else:
        return '00-8'


final_df["HOUR"] = (final_df["MINUTE"] / 60).astype(int)

final_df['fascia_oraria'] = final_df['HOUR'].apply(fascia_oraria)

del final_df["HOUR"]
final_df

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8992\4279114958.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["HOUR"] = (final_df["MINUTE"] / 60).astype(int)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8992\4279114958.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['fascia_oraria'] = final_df['HOUR'].apply(fascia_oraria)


,SQUAREID,RAIN,MONTH,DAY,MINUTE,WEEK_DAY,IS_RAIN,W_POWER,fascia_oraria
0,4037,NaN,11,1,0,4,NaN,32348.159136,00-8
1,4154,NaN,11,1,0,4,NaN,32348.159136,00-8
2,4155,NaN,11,1,0,4,NaN,32348.159136,00-8
3,4156,NaN,11,1,0,4,NaN,32348.159136,00-8
4,4269,NaN,11,1,0,4,NaN,32348.159136,00-8
...,...,...,...,...,...,...,...,...,...
22227259,8502,NaN,12,31,1430,1,NaN,65469.601728,19-00
22227260,8503,NaN,12,31,1430,1,NaN,65469.601728,19-00
22227261,8621,NaN,12,31,1430,1,NaN,21823.200576,19-00
22227262,8972,NaN,12,31,1430,1,NaN,65469.601728,19-00


Eliminiamo qualche dato che non ci serve più (altrimenti il pc esplode e mi picchia)

In [34]:
del power_time_dec_df, power_time_nov_df, precip_avail_df, precip_df

Creiamo un geodataframe che mi dica a quali comuni appartengono le varie **SQUAREID**.  
Procediamo in maniera molto simile a come avevamo fatto nei precedenti *EDA* per novembre e dicembre, ma aggiungiamo i comuni. Possiamo farlo aggiungendo la funzione `predicate='intersects'` che ci permette di fare un'intersezione tra le geometrie dei quadratini e qualle dei comuni.

In [58]:
# importiamo i files
gdf_com = gp.read_file(file_path_ext + 'gdf_com/gdf_com.shp')
tn_grid_gdf = gp.read_file(file_path_raw + "trentino-grid.geojson")

# filtriamo solo le regioni amministrative del Trentino
filtro = (gdf_com['COD_REG']==4) & (gdf_com['COD_PROV'] == 22) 
gdf_temp = gdf_com[filtro]
del(gdf_com)
gdf_com = gdf_temp
del(gdf_temp)
crs_comune = 'EPSG:4326'
gdf_com_rep = gdf_com[filtro].to_crs(crs_comune)

# facciamo un merge usando predicate='intersects' per intersecare le geometrie dei comuni
gdf_celle_com= gp.sjoin(tn_grid_gdf, gdf_com_rep, how='left', predicate='intersects') 

# teniamoci solo i dati che ci interessano 
cols_to_keep = ['cellId','COMUNE']
gdf_celle_com = gdf_celle_com.drop(columns = [col for col in gdf_celle_com.columns if col not in cols_to_keep])

gdf_celle_com


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\geopandas\geodataframe.py:1750: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


,cellId,COMUNE
0,38,Avio
1,39,Avio
2,40,Avio
3,154,Avio
4,155,Avio
...,...,...
6570,11336,Canazei
6571,11337,Canazei
6572,11452,Canazei
6573,11453,Canazei


In [63]:
temp_df = pd.merge(final_df,gdf_celle_com, left_on="SQUAREID", right_on="cellId",how="left")
temp_df

,SQUAREID,RAIN,MONTH,DAY,MINUTE,WEEK_DAY,IS_RAIN,W_POWER,fascia_oraria,cellId,COMUNE
0,4037,NaN,11,1,0,4,NaN,32348.159136,00-8,4037,Vattaro
1,4037,NaN,11,1,0,4,NaN,32348.159136,00-8,4037,Bosentino
2,4154,NaN,11,1,0,4,NaN,32348.159136,00-8,4154,Vattaro
3,4154,NaN,11,1,0,4,NaN,32348.159136,00-8,4154,Bosentino
4,4154,NaN,11,1,0,4,NaN,32348.159136,00-8,4154,Vigolo Vattaro
...,...,...,...,...,...,...,...,...,...,...,...
26158969,8503,NaN,12,31,1430,1,NaN,65469.601728,19-00,8503,Varena
26158970,8621,NaN,12,31,1430,1,NaN,21823.200576,19-00,8621,Tesero
26158971,8621,NaN,12,31,1430,1,NaN,21823.200576,19-00,8621,Varena
26158972,8972,NaN,12,31,1430,1,NaN,65469.601728,19-00,8972,Varena


filtriamo i dati solo per il comune di Trento:

In [65]:
comuni = ["Trento", "Rovereto"]
temp_df = temp_df[temp_df["COMUNE"].isin(comuni)]

final_df = temp_df[["SQUAREID", "COMUNE", "MONTH", "DAY", "WEEK_DAY", "MINUTE", "fascia_oraria", "W_POWER", "RAIN", "IS_RAIN"]]
final_df

,SQUAREID,COMUNE,MONTH,DAY,WEEK_DAY,MINUTE,fascia_oraria,W_POWER,RAIN,IS_RAIN
4327504,5317,Trento,11,1,4,0,00-8,2556.602048,NaN,NaN
4327505,5433,Trento,11,1,4,0,00-8,19362.051510,3.0,True
4327506,5434,Trento,11,1,4,0,00-8,1138.022534,NaN,NaN
4327507,5549,Trento,11,1,4,0,00-8,18005.142240,NaN,NaN
4327508,5550,Trento,11,1,4,0,00-8,8243.435393,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
25392302,6253,Trento,12,31,1,1420,19-00,11919.600000,NaN,NaN
25392305,6018,Trento,12,31,1,1430,19-00,11830.853520,NaN,NaN
25392307,6019,Trento,12,31,1,1430,19-00,14087.858332,NaN,NaN
25392310,6136,Trento,12,31,1,1430,19-00,24346.191359,NaN,NaN


gli ultimi ritocchi

In [71]:
final_df = final_df.reset_index()
del final_df["index"]

final_df['RAIN'] = final_df['RAIN'].fillna(0).astype(np.int64)
del final_df["IS_RAIN"]

final_df

,SQUAREID,COMUNE,MONTH,DAY,WEEK_DAY,MINUTE,fascia_oraria,W_POWER,RAIN
0,5317,Trento,11,1,4,0,00-8,2556.602048,0
1,5433,Trento,11,1,4,0,00-8,19362.051510,3
2,5434,Trento,11,1,4,0,00-8,1138.022534,0
3,5549,Trento,11,1,4,0,00-8,18005.142240,0
4,5550,Trento,11,1,4,0,00-8,8243.435393,0
...,...,...,...,...,...,...,...,...,...
2649315,6253,Trento,12,31,1,1420,19-00,11919.600000,0
2649316,6018,Trento,12,31,1,1430,19-00,11830.853520,0
2649317,6019,Trento,12,31,1,1430,19-00,14087.858332,0
2649318,6136,Trento,12,31,1,1430,19-00,24346.191359,0


Salviamo il file finale 

In [72]:
final_df.to_csv(file_path_save + "final_df.csv", index=False)